In [7]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.log4j._


In [8]:
val spark = SparkSession.builder().getOrCreate()
val data = spark.read.option("header", "true")
            .option("inferSchema", "true")
            .format("csv")
            .load("../datasets/Wholesale customers data.csv")
data.show()

+-------+------+-----+-----+-------+------+----------------+----------+
|Channel|Region|Fresh| Milk|Grocery|Frozen|Detergents_Paper|Delicassen|
+-------+------+-----+-----+-------+------+----------------+----------+
|      2|     3|12669| 9656|   7561|   214|            2674|      1338|
|      2|     3| 7057| 9810|   9568|  1762|            3293|      1776|
|      2|     3| 6353| 8808|   7684|  2405|            3516|      7844|
|      1|     3|13265| 1196|   4221|  6404|             507|      1788|
|      2|     3|22615| 5410|   7198|  3915|            1777|      5185|
|      2|     3| 9413| 8259|   5126|   666|            1795|      1451|
|      2|     3|12126| 3199|   6975|   480|            3140|       545|
|      2|     3| 7579| 4956|   9426|  1669|            3321|      2566|
|      1|     3| 5963| 3648|   6192|   425|            1716|       750|
|      2|     3| 6006|11093|  18881|  1159|            7425|      2098|
|      2|     3| 3366| 5403|  12974|  4400|            5977|    

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3412cde4
data: org.apache.spark.sql.DataFrame = [Channel: int, Region: int ... 6 more fields]


In [9]:
data.printSchema()

root
 |-- Channel: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- Fresh: integer (nullable = true)
 |-- Milk: integer (nullable = true)
 |-- Grocery: integer (nullable = true)
 |-- Frozen: integer (nullable = true)
 |-- Detergents_Paper: integer (nullable = true)
 |-- Delicassen: integer (nullable = true)



In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+
|summary|           Channel|            Region|             Fresh|              Milk|          Grocery|           Frozen|  Detergents_Paper|        Delicassen|
+-------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+
|  count|               440|               440|               440|               440|              440|              440|               440|               440|
|   mean|1.3227272727272728| 2.543181818181818|12000.297727272728| 5796.265909090909|7951.277272727273|3071.931818181818|2881.4931818181817|1524.8704545454545|
| stddev|0.4680515694791137|0.7742724492301002|12647.328865076885|7380.3771745708445|9503.162828994346|4854.673332592367| 4767.854447904201|2820.1059373693965|
|    min|                 1|            

In [12]:
val feature_data = data.select($"Fresh", $"Milk", $"Grocery", $"Frozen", $"Detergents_Paper", $"Delicassen")

feature_data: org.apache.spark.sql.DataFrame = [Fresh: int, Milk: int ... 4 more fields]


In [11]:
val assembler = new VectorAssembler()
                .setInputCols(Array("Fresh", "Milk", "Grocery", "Frozen", "Detergents_Paper", "Delicassen"))
                .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_4a2fa788a35c, handleInvalid=error, numInputCols=6


In [17]:
val training_data = assembler.transform(feature_data).select("features")

training_data: org.apache.spark.sql.DataFrame = [features: vector]


In [16]:
val kmeans = new KMeans().setK(3).setSeed(42)

kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_ea4f2ccf97e3


In [19]:
val model = kmeans.fit(training_data)

model: org.apache.spark.ml.clustering.KMeansModel = KMeansModel: uid=kmeans_ea4f2ccf97e3, k=3, distanceMeasure=euclidean, numFeatures=6


In [20]:
val summary = model.summary
summary.predictions.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[12669.0,9656.0,7...|         0|
|[7057.0,9810.0,95...|         0|
|[6353.0,8808.0,76...|         0|
|[13265.0,1196.0,4...|         0|
|[22615.0,5410.0,7...|         1|
|[9413.0,8259.0,51...|         0|
|[12126.0,3199.0,6...|         0|
|[7579.0,4956.0,94...|         0|
|[5963.0,3648.0,61...|         0|
|[6006.0,11093.0,1...|         2|
|[3366.0,5403.0,12...|         0|
|[13146.0,1124.0,4...|         0|
|[31714.0,12319.0,...|         1|
|[21217.0,6208.0,1...|         0|
|[24653.0,9465.0,1...|         1|
|[10253.0,1114.0,3...|         0|
|[1020.0,8816.0,12...|         0|
|[5876.0,6157.0,29...|         0|
|[18601.0,6327.0,1...|         0|
|[7780.0,2495.0,94...|         0|
+--------------------+----------+
only showing top 20 rows



summary: org.apache.spark.ml.clustering.KMeansSummary = org.apache.spark.ml.clustering.KMeansSummary@7c14adf1
